In [1]:
import pandas as pd
import numpy as np
import json 
import re
import glob
import nltk

In [2]:
df = pd.read_csv('atla_lok_transcript.csv',index_col=0)
df['dialogue'] = df['dialogue'].astype(str)
atla_df = df[df.series==1]
lok_df  = df[df.series==2]

In [3]:
df['combined_dialog']      = df.groupby(by='character_name')['dialogue'].transform(lambda x: ' '.join(x))
lok_df['combined_dialog']  = lok_df.groupby(by='character_name')['dialogue'].transform(lambda x: ' '.join(x))
atla_df['combined_dialog'] = atla_df.groupby(by='character_name')['dialogue'].transform(lambda x: ' '.join(x))

df = df[['character_name','combined_dialog']]
df = df.drop_duplicates(subset=['character_name'],keep="first")

atla_per_char = atla_df[['character_name','combined_dialog']]
atla_per_char = atla_per_char.drop_duplicates(subset=['character_name'],keep="first")

lok_per_char  = lok_df[['character_name','combined_dialog']]
lok_per_char  = lok_per_char.drop_duplicates(subset=['character_name'],keep="first")


/Users/philip/opt/anaconda3/envs/socialgraphs/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/philip/opt/anaconda3/envs/socialgraphs/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
def split_into_list(text):
    text = str(text)
    return nltk.tokenize.sent_tokenize(text)

#Split text into lists of sentences
df['combined_dialog'] = df['combined_dialog'].apply(lambda x: split_into_list(x))
atla_per_char['combined_dialog'] = atla_per_char['combined_dialog'].apply(lambda x: split_into_list(x))
lok_per_char['combined_dialog']  = lok_per_char['combined_dialog'].apply(lambda x: split_into_list(x))

#Rename column to Name
df.rename(columns={'character_name': 'Name'},inplace=True)
atla_per_char.rename(columns={'character_name': 'Name'},inplace=True)
lok_per_char.rename(columns={'character_name': 'Name'},inplace=True)

#Set Name column as the index
df = df.set_index("Name")
atla_per_char = atla_per_char.set_index("Name")
lok_per_char = lok_per_char.set_index("Name")

In [6]:
atla_per_char.to_json("Dialogs_atla.json",orient='index')
lok_per_char.to_json("Dialogs_lok.json",orient='index')
df.to_json("Dialogs_both.json",orient='index')

In [7]:
#Check number of characters in the graph data that have dialogs

data = pd.read_csv('avatar_data.csv',index_col=0)
common_names = len(np.intersect1d(data.name, df.index))
print("The number of common names between dialogs and graph data is : ",common_names)

The number of common names between dialogs and graph data is :  252
